In [109]:
import pandas as pd
import re
sheet = pd.read_excel('../../output/山东省千佛山医院病理科肺癌数据.xls',dtype = str)

In [110]:
sheet.head(2)

,病理号,住院号,性别,年龄,标本名称,病理诊断,肉眼所见,临床诊断,诊断时间,诊断医生,居住地,肿瘤家族史,吸烟史,术前辅助治疗,术后辅助治疗,pTNM分期,预后
0,201313341,188019,男,71岁,肺,（左肺上叶）鳞状细胞癌（中度分化），体积4.5*3.5*2.5cm，侵达脏层胸膜，残端切线未...,（左肺上叶）肺组织一块，体积20*14*3.5cm，表面大部分光滑，覆脏层胸膜，肺门部粗糙，...,左上肺占位,2013-10-25 10:54:33,刘群,山东省济南市,母亲有子宫癌病史，二姐因舌癌去世，三弟有直肠癌病史,30年，20支/日，已戒烟20年,无,奥沙利铂+培美曲塞二钠化疗,鳞癌，pT2N0MO.Ib期,骨、肝转移
1,201313542,244707,男,61岁,肺,（右中叶）肺鳞状细胞癌（Ⅱ级），部分区域坏死，体积4*3.5*2cm。\n ...,右肺中叶，体积11*10*2.5cm，表面大部分光滑覆脏层胸膜，肺门部粗糙，距支气管残端0....,肺内阴影（右中）,2013-10-29 17:03:39,崔晶,山东省高青县,无,30年，20支/日,无,地行奈达铂+吉西他滨化疗,鳞癌，pT2aN1MO.IIa期,nan


In [111]:
def preprocess(input_str):
    res = re.sub('非小细胞癌','',input_str)
    return res

In [112]:
def strQ2B(ustring):
    """全角转半角"""
    ustring = str(ustring)
    rstring = ""
    for uchar in ustring:
        inside_code = ord(uchar)
        if inside_code == 12288:  # 全角空格直接转换
            inside_code = 32
        elif (inside_code >= 65281 and inside_code <= 65374):  # 全角字符（除空格）根据关系转化
            inside_code -= 65248
        rstring += chr(inside_code)
    return rstring

In [113]:
pattern1 = re.compile(r'混合[性型].{0,5}癌',re.M) 
pattern2 = re.compile(r'小细胞癌',re.M)
pattern3 = re.compile(r'大细胞癌|大细胞神经内分泌癌',re.M)
pattern4 = re.compile(r'鳞癌|鳞状细胞癌',re.M)
pattern5 = re.compile(r'腺癌',re.M)
pattern_fq = re.compile(r'(.*?)([^\u4E00-\u9FA5]*[期]*)',re.M)
pattern_fq2 = re.compile(r'([^a-zA-Z0-9]*)([^\u4E00-\u9FA5]*[期]*)',re.M)
pattern_chinese = re.compile(r'[\u4E00-\u9FA5]+',re.M)

In [114]:
def search_1(text):
    if pattern1.search(text):
        return True
    if pattern4.search(text) and pattern5.search(text):
        return True
    if pattern5.search(text) and pattern2.search(text):
        return True
    if pattern4.search(text) and pattern2.search(text):
        return True
    return False

In [115]:
def judge(text):
    text = preprocess(text)
    res = ''
    if search_1(text):
        res = '混合型癌'
        return res
    if pattern2.search(text):
        res = '小细胞癌'
        return res
    if pattern3.search(text):
        res = '大细胞癌'
        return res
    if pattern4.search(text):
        res = '鳞癌'
        return res
    if pattern5.search(text):
        res = '腺癌'
        return res
    res = '未知'
    return res


In [116]:
def judge2(text):
    text = strQ2B(text)
    seqs = text.split(',')
    if len(seqs) >= 2:
        if pattern_chinese.search(seqs[0]):
            seqs = seqs[1:]
    text = ''.join(seqs)
    res = '未知'
    if pattern_fq.search(text):
        res = pattern_fq.search(text).group()
        if res == '':
            if pattern_fq2.search(text):
                res = pattern_fq2.search(text).group(2)
        if res == 'nan' or res == '':
            res = '未知'
        
    return res     

In [117]:
test_str = 'T2N2M0，Ⅲa'
print(judge2(test_str))

T2N2M0Ⅲa


In [118]:
test_str = '混合性癌（小细胞癌，鳞癌）T3N0MO.IIa期'
print(judge2(test_str))
print(pattern_fq.search(test_str).group(0))
print(pattern_fq.search(test_str))

T3N0MO.IIa期

<_sre.SRE_Match object; span=(0, 0), match=''>


In [119]:
print(re.match(r'^[\u4E00-\u9FA5]+', '免疫组化'))

<_sre.SRE_Match object; span=(0, 4), match='免疫组化'>


In [120]:
print(re.match(r'[^\u4E00-\u9FA5]+', '免疫组化'))

None


In [121]:
print(re.match(r'[[^\u4E00-\u9FA5]期]+', '免疫组化期'))

None


In [122]:
print(re.match(r'[[^\u4E00-\u9FA5]期]+', '免疫组化期'))

None


In [123]:
print(re.match(r'[^\u4E00-\u671E\u6720-\u9FA5]+', '免疫组化期'))

None


In [124]:
print(re.match(r'[^\u4E00-\u671E\u6720-\u9FA5]*', '免疫组化期'))

<_sre.SRE_Match object; span=(0, 0), match=''>


In [125]:
print(re.match(r'[^\u4E00-\u671E|^\u6720-\u9FA5]*', '免疫组化期'))

<_sre.SRE_Match object; span=(0, 0), match=''>


In [126]:
print(re.match(r'[^\u4E00-\u671E]+', '免疫组化期'))

None


In [127]:
print(re.match(r'[^\u4E00]+', '免疫组化期'))

<_sre.SRE_Match object; span=(0, 5), match='免疫组化期'>


In [128]:
print(re.match(r'[^\u4E00-\u6710|期]+', '免疫组化期'))

None


In [129]:
print(re.match(r'[^\u4E00-\u6710]|期+', '免疫组化期'))

None


In [130]:
print(re.match(r'[^\u4E00-\u6710]+[期].', '免疫组化期'))

None


In [131]:
print(re.search(r'[^\u4E00-\u6710]+[期]*', '免疫组化期'))

<_sre.SRE_Match object; span=(1, 3), match='疫组'>


In [132]:
print(re.match(r'(.*)[^\u4E00-\u9FA5]*[期]*', '免疫组化期'))

<_sre.SRE_Match object; span=(0, 5), match='免疫组化期'>


In [133]:
print(re.match(r'[^\u4E00-\u9FA5]*[期]+', '免疫组化期'))

None


In [134]:
print(re.search(r'[^\u4E00-\u9FA5]*[期]+', '免疫组化期'))
print(re.search(r'[^\u4E00-\u9FA5]*[期]*', '免疫组化期')) # search 返回的是第一个匹配,这样写的正则 空字符串也符合条件

<_sre.SRE_Match object; span=(4, 5), match='期'>
<_sre.SRE_Match object; span=(0, 0), match=''>


In [135]:
print(re.search(r'[^\u4E00-\u9FA5\s]*[期]+', '免疫组化期'))
print(re.search(r'[^\u4E00-\u9FA5\S]*[期]*', '免疫组化期')) 

<_sre.SRE_Match object; span=(4, 5), match='期'>
<_sre.SRE_Match object; span=(0, 0), match=''>


In [136]:
print(re.search(r'[^\u4E00-\u9FA5\s]*[期]+', '免疫组化期'))
print(re.search(r'[^\u4E00-\u9FA5\s]*[期\S]*', '免疫组化期'))

<_sre.SRE_Match object; span=(4, 5), match='期'>
<_sre.SRE_Match object; span=(0, 5), match='免疫组化期'>


In [137]:
print(re.search(r'[^\u4E00-\u9FA5]+[期]+|[^\u4E00-\u9FA5]*', '免疫组化期'))

<_sre.SRE_Match object; span=(0, 0), match=''>


In [138]:
print(re.search(r'[\u671F]+', '免疫组化期')) 

<_sre.SRE_Match object; span=(4, 5), match='期'>


In [139]:
'期'.encode()

b'\xe6\x9c\x9f'

In [140]:
'期'.encode('utf-8')

b'\xe6\x9c\x9f'

In [141]:
print(u'期')

期


In [142]:
ord('期') # 代表了第几个unicode字符

26399

In [143]:
hex(ord('期'))

'0x671f'

In [144]:
hex(ord('期')).upper().replace('0X', r'\u')

'\\u671F'

In [145]:
print(hex(ord('期')).upper().replace('0X', r'\u'))

\u671F


In [146]:
print(re.search(r'[^\u4E00-\u9FA5]*[期]+', 'pT2aN0M0 Ⅰb期'))
print(re.search(r'[^\u4E00-\u9FA5]*[期]*', 'pT2aN0M0 Ⅰb期'))

<_sre.SRE_Match object; span=(0, 12), match='pT2aN0M0 Ⅰb期'>
<_sre.SRE_Match object; span=(0, 12), match='pT2aN0M0 Ⅰb期'>


In [147]:
print(re.search(r'[^\u4E00-\u9FA5]*[期]*', 'pT2N3M1未手术'))

<_sre.SRE_Match object; span=(0, 7), match='pT2N3M1'>


In [148]:
print(re.search(r'(.*?)([^\u4E00-\u9FA5]*[期]*)', 'pT2N3M1未手术'))

<_sre.SRE_Match object; span=(0, 7), match='pT2N3M1'>


In [149]:
print(re.search(r'(.*?)([^\u4E00-\u9FA5]*[期]*)', 'pT2aN0M0 Ⅰb期'))
print(re.search(r'(.*?)([^\u4E00-\u9FA5]*[期]*)', 'pT2aN0M0 Ⅰb期').group())

<_sre.SRE_Match object; span=(0, 12), match='pT2aN0M0 Ⅰb期'>
pT2aN0M0 Ⅰb期


这样可以保留"期"

[^...]	不在[]中的字符：[^abc] 匹配除了a,b,c之外的字符。

In [150]:
sheet.shape

(448, 17)

In [151]:
row_num = sheet.shape[0]
row_num

448

In [152]:
azlx_list = []
fq_list = []
blzd_texts = sheet['病理诊断']
fq_texts = sheet['pTNM分期']
for idx in range(row_num):
    res = judge(fq_texts[idx])
    if res == '未知':
        res = judge(blzd_texts[idx])
    azlx_list.append(res)
    fq = judge2(fq_texts[idx])
    fq_list.append(fq)
    

In [153]:
len(azlx_list)

448

In [154]:
azlx_list[:5]

['鳞癌', '鳞癌', '鳞癌', '腺癌', '鳞癌']

In [155]:
len(fq_list)

448

In [156]:
fq_list[:5]

['pT2N0MO.Ib期',
 'pT2aN1MO.IIa期',
 'pT2bN1MO.IIb期',
 'T1aNOMO.Ia期',
 'pT2aN0MO.Ib期']

In [157]:
sheet['癌种类型'] = azlx_list
sheet['分期类型'] = fq_list

In [158]:
sheet.head(5)

,病理号,住院号,性别,年龄,标本名称,病理诊断,肉眼所见,临床诊断,诊断时间,诊断医生,居住地,肿瘤家族史,吸烟史,术前辅助治疗,术后辅助治疗,pTNM分期,预后,癌种类型,分期类型
0,201313341,188019,男,71岁,肺,（左肺上叶）鳞状细胞癌（中度分化），体积4.5*3.5*2.5cm，侵达脏层胸膜，残端切线未...,（左肺上叶）肺组织一块，体积20*14*3.5cm，表面大部分光滑，覆脏层胸膜，肺门部粗糙，...,左上肺占位,2013-10-25 10:54:33,刘群,山东省济南市,母亲有子宫癌病史，二姐因舌癌去世，三弟有直肠癌病史,30年，20支/日，已戒烟20年,无,奥沙利铂+培美曲塞二钠化疗,鳞癌，pT2N0MO.Ib期,骨、肝转移,鳞癌,pT2N0MO.Ib期
1,201313542,244707,男,61岁,肺,（右中叶）肺鳞状细胞癌（Ⅱ级），部分区域坏死，体积4*3.5*2cm。\n ...,右肺中叶，体积11*10*2.5cm，表面大部分光滑覆脏层胸膜，肺门部粗糙，距支气管残端0....,肺内阴影（右中）,2013-10-29 17:03:39,崔晶,山东省高青县,无,30年，20支/日,无,地行奈达铂+吉西他滨化疗,鳞癌，pT2aN1MO.IIa期,nan,鳞癌,pT2aN1MO.IIa期
2,201315170,197718,男,64岁,肺,（左肺下叶）鳞状细胞癌（中度分化），体积5.5*3.5*3cm。\n ...,左肺下叶，体积16*9*3cm，表面大部分光滑覆脏层胸膜，肺门部粗糙，打开各段支气管及做肺切...,肺部阴影,2013-11-28 15:25:07,赫淑倩,山东省济南市,无,30年，20支/日,无,奈达铂+吉西他滨化疗,鳞癌，pT2bN1MO.IIb期,nan,鳞癌,pT2bN1MO.IIb期
3,201315241,247434,女,57岁,肺,（右肺上叶）腺癌（中度分化），体积1.7*1.4*1cm，侵达胸膜。\n ...,右肺上叶，体积17.5*10*4cm，表面大部分光滑覆脏层胸膜，肺门部粗糙，打开各段支气管及...,右肺上叶占位,2013-11-27 15:30:36,赫淑倩,山东省淄博市,无,无,无,奥沙利铂+培美曲塞二钠化疗、奥沙利铂+泽菲化疗,腺癌，T1aNOMO.Ia期,nan,腺癌,T1aNOMO.Ia期
4,201401024,250398,男,56岁,肺,左肺上叶鳞状细胞癌（中度分化），体积3*2*2cm，未累及支气管残端切线。\n肺门（4个）、...,（左肺上叶）肺组织，体积10*9*13*2.5cm，表面光滑覆脏层胸膜，打开各支气管及做肺切...,左肺癌,2014-01-24 16:54:38,周萍,山东省济南市,父亲死于食道癌,30年，40支/日,无,奥沙利铂+吉西他滨化疗,鳞癌，pT2aN0MO.Ib期,nan,鳞癌,pT2aN0MO.Ib期


In [159]:
sheet.to_excel('../../output/山东省千佛山医院病理科肺癌数据_count.xls',encoding = 'utf_8_sig',index = False)

In [3]:
import re   
pattern = re.compile(r'ALK((^ALK){0,16}?)',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>



In [4]:
import re   
pattern = re.compile(r'ALK([^(ALK)]{0,16}?)',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>




去除/不包括 某一词而不是某个字母    
https://stackoverflow.com/questions/2078915/a-regular-expression-to-exclude-a-word-string





(?! re)	前向否定界定符。与肯定界定符相反；当所含表达式不能在字符串当前位置匹配时成功.

?! 表示这里的括号不是分组的意思，且是否定。






In [9]:
import re   
pattern = re.compile(r'ALK(.{0,16}(?!ALK))',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 26), match='ALK(D5F3阴性对照)（-）ALK'>
(d5f3阴性对照)（-）alk


In [11]:
import re   
pattern = re.compile(r'ALK(.{0,16}(?!ALK$))',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 26), match='ALK(D5F3阴性对照)（-）ALK'>
(d5f3阴性对照)（-）alk


In [26]:
import re   
pattern = re.compile(r'ALK([(?!ALK)]{0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 11), match='ALK('>
(
<_sre.SRE_Match object; span=(23, 27), match='ALK('>
(


In [30]:
import re   
pattern = re.compile(r'ALK(.*?)',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>



In [31]:
import re   
pattern = re.compile(r'ALK((?!^ALK){0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>



In [32]:
import re   
pattern = re.compile(r'ALK((?!^ALK?){0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>



In [33]:
import re   
pattern = re.compile(r'ALK((?!\bALK\b){0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>

<_sre.SRE_Match object; span=(23, 26), match='ALK'>



In [34]:
import re   
pattern = re.compile(r'ALK(.+(?!\bALK\b){0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 52), match='ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'>
(d5f3阴性对照)（-）alk(d5f3)(ventana全自动免疫组化)(-)。


In [ ]:
import re   
pattern = re.compile(r'ALK(.+(?!\bALK\b){0,13})',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 
text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

如果末尾要不为ALK，那么即使给.加了?非贪婪匹配，那么 .会把ALK吸收进去。

In [37]:
import re   
pattern = re.compile(r'ALK(.+(?!ALK)+?)',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group(1)
        print(match)
        window = window.lower()
        yield window 
text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 52), match='ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'>
(d5f3阴性对照)（-）alk(d5f3)(ventana全自动免疫组化)(-)。


如何让下一个ALK不被吸收进去？


the regex engine should enter and leave the group only once.

一个()代表的group相当于正则引擎进入一次离开一次，然后把进入的地方和离开的地方的中间地带记录下来。


翻了很多帖子，暂时没有找到解决方案

In [44]:
import re   
pattern = re.compile(r'ALK',re.I|re.M) #忽略大小写
def get_windows(input_str):
    for match in pattern.finditer(input_str):
        window = match.group()
        print(match)
        print(match.span(0)[0])
        print(input_str[match.span()[0]])
        window = window.lower()
        yield window 
text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)

<_sre.SRE_Match object; span=(7, 10), match='ALK'>
7
A
alk
<_sre.SRE_Match object; span=(23, 26), match='ALK'>
23
A
alk


In [1]:
s = 'aeajeaeja'
print(s[:100])

aeajeaeja


In [14]:
import re   
pattern = re.compile(r'ALK',re.I|re.M) #忽略大小写
def get_windows(input_str):
    matches = list(pattern.finditer(input_str))
    match_num = len(matches)
    if match_num == 1:
        start = matches[0].span(0)[1]
        window = input_str[start:start+13]
        yield window 
    elif match_num > 1:
        for idx in range(match_num-1):
            start = matches[idx].span(0)[1]
            next_pos = matches[idx+1].span(0)[0]
            end = start + 16 if start + 16 < next_pos else next_pos
            window = input_str[start:end]
            yield window
        start = matches[idx+1].span(0)[1]
        window = input_str[start:start+16]
        yield window

text = '免疫组化结果：ALK(D5F3阴性对照)（-）ALK(D5F3)(ventana全自动免疫组化)(-)。'
for window in get_windows(text):
    print(window)
text = '没有'
if not get_windows(text):
    print('没有')
else:
    print(get_windows(text))
    
    
if not len(list(get_windows(text))):
    print('没有')
else:
    print(get_windows(text))
    
    
    
    
if not list(get_windows(text)):
    print('没有')
else:
    print(get_windows(text))

(D5F3阴性对照)（-）
(D5F3)(ventana全自
<generator object get_windows at 0x00000190CA3E6150>
没有
没有


In [15]:
import re 
text = '建议不要做alk检测，'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 11), match='建议不要做alk检测，'>


In [16]:
text = '建议不要做alk检测'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 10), match='建议不要做alk检测'>


In [17]:
text = '建议不要做alk检,eaea'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 10), match='建议不要做alk检,'>


In [18]:
text = '建议不要做alk检,eaea'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)+',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 14), match='建议不要做alk检,eaea'>


In [20]:
text = '建议不要做检eaea'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)+',re.M | re.I)
print(pattern.search(text))

None


In [21]:
text = '建议不要做alk检eaea,'
pattern = re.compile('.*建议.*ALK[^,.;]*(,|.|;)?',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 14), match='建议不要做alk检eaea,'>


In [27]:
text = '建议不要做alk检eaea'
pattern = re.compile('.*建议.*ALK[^,.;]+[,|.|;]+?',re.M | re.I)
print(pattern.search(text))

None


In [28]:
text = '建议不要做alk,'
pattern = re.compile('.*建议.*ALK[^,.;]*[,|.|;]?',re.M | re.I)
print(pattern.search(text))

<_sre.SRE_Match object; span=(0, 9), match='建议不要做alk,'>


In [30]:
text = '建议不要做alk'
pattern = re.compile('.*建议.*ALK[^,.;]*[,|.|;]+?',re.M | re.I)
print(pattern.search(text))

None
